# Define Spatial Extent of Project

This notebook retrieves Metropolitan Council's 2020 boundaries of the Twin Cities area and extracts the Minneapolis's boundary and buffers 8 kilometers (large emissions were [shown to travel this distance](https://plumepgh.org/model_data.html) if the wind blows right). Both Minneapolis and the Extent are saved in geodatabase.

Output Coordinate Reference System: UTM 15N - EPSG:26915

Source: https://gisdata.mn.gov/dataset/us-mn-state-metc-bdry-census2020counties-ctus

Metadata: https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/bdry_census2020counties_ctus/metadata/metadata.html


In [1]:
### Import Packages

# File manipulation

import os # For working with Operating System
import shutil # For deleting folders
import urllib # For accessing websites
import zipfile # For extracting from Zipfiles
from io import BytesIO # For reading bytes objects

# Database 

import psycopg2

# Analysis

import arcpy
import pandas as pd

# Get CWD

cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

# Create GeoDataBase
# This is the communal GeoDataBase, only run once

if not os.path.exists(os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')): # If it doesn't exist, create it

    arcpy.management.CreateFileGDB(os.path.join(cwd, '..', '..', 'data'), 'QAQC')

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

In [2]:
### Definitions

def extract_zip_from_url(url=None, path=None):
    '''Extract a zipfile from the internet
    then unpack it in to it's own folder 
    within the working directory.
    Takes a single url (string).'''

    if not os.path.exists(path):
        os.mkdir(path)
    # Unload zip into the new folder
    response = urllib.request.urlopen(url) # Get a response
    zip_folder = zipfile.ZipFile(BytesIO(response.read())) # Read Response
    zip_folder.extractall(path=path) # Extract files
    zip_folder.close() # Close zip object

In [3]:
# Download Data

## Twin Cities Metro Boundaries - Downloaded from MN GeospatialCommons gisdata.mn.gov  (~ 5mb)

url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/bdry_census2020counties_ctus/shp_bdry_census2020counties_ctus.zip"

# Create folder name for file
folder_name = url.split('/')[-1][:-4]
# Make folder for files
savepath = os.path.join(cwd, '..','..','data', folder_name)

extract_zip_from_url(url, savepath)

In [4]:
# Read & Select

# Get path

filename = 'Census2020CTUs.shp'
path = os.path.join(savepath, filename)

arcpy.MakeFeatureLayer_management(path, "TCMA_lyr")

# Select Minneapolis

mpls_boundary = arcpy.management.SelectLayerByAttribute("TCMA_lyr", "SUBSET_SELECTION",
                                                        arcpy.AddFieldDelimiters(datasource='TCMA_lyr', field= 'CTU_NAME') + "= 'Minneapolis'")

# Write the selected features to a new featureclass
arcpy.management.CopyFeatures(mpls_boundary, "mpls_boundary")

<Result 'D:\\QualityAirQualityCities\\arcpy\\qaqc\\..\\..\\data\\QAQC.gdb\\mpls_boundary'>

In [5]:
# Buffer

arcpy.analysis.Buffer('mpls_boundary', 'mpls_8km', '8 Kilometers')

# # Save as a geojson (Don't need)

# arcpy.conversion.FeaturesToJSON(mpls_boundary, 'mpls_boundary.geojson', geoJSON='GEOJSON')

<Result 'D:\\QualityAirQualityCities\\arcpy\\qaqc\\..\\..\\data\\QAQC.gdb\\mpls_8km'>

## For Database

In [8]:
# Upload to Database

# Get credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].rstrip('\n').split(', ')
    
# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

conn = psycopg2.connect(**pg_connection_dict)

# Create Cursor for commands

cur = conn.cursor()

# Insert into table

cols = ['CTU_ID', 'CTU_NAME', 'CTU_CODE', 'SHAPE@WKT'] # Relative columns

cursor = arcpy.da.SearchCursor(mpls_boundary, cols)

for row in cursor:
    
    cur.execute(
        'INSERT INTO Minneapolis_Boundary(CTU_ID, CTU_NAME, CTU_CODE, geometry)'
        'VALUES (%(ctu_id)s, %(ctu_name)s, %(ctu_code)s, ST_Transform(ST_SetSRID(ST_GeomFromText(%(geom)s), 26915),4326)::geometry);',
        {'ctu_id': row[0],
         'ctu_name' : row[1],
         'ctu_code': row[2],
         'geom': row[3]})

    # Commit command

    conn.commit()

# Close cursor

cur.close()

# Close connection

conn.close()